# Ensemble methods
In the realm of machine learning, the age-old adage, 'Unity is strength,' resonates profoundly, encapsulating the fundamental principle governing the remarkably potent 'ensemble methods.' These methods harness the collective power of multiple learning models, harmoniously working together to achieve remarkable outcomes.

In this notebook, different ensemble models and some concepts will be explaines in details. This educational notebook is based on this <a href="https://towardsdatascience.com/ensemble-methods-bagging-boosting-and-stacking-c9214a10a205" target="_blank">link</a>.

## Introduction
Ensemble learning represents a powerful machine learning paradigm wherein numerous models, often referred to as "weak learners," are individually trained to tackle the same problem and subsequently integrated to yield enhanced results. The central hypothesis underlying this approach is that through the proper amalgamation of these weak models, we can attain significantly improved accuracy and/or robustness in the final model.

The most fundamental traits desired in a model are low bias and low variance, even though they frequently exhibit opposite tendencies. Achieving a balance between these two attributes is crucial for effectively addressing a problem. On one hand, the model should possess sufficient degrees of freedom to adequately capture the intricate complexities of the data at hand, enabling it to "solve" the problem. On the other hand, excessive degrees of freedom must be avoided to prevent high variance and ensure robustness. This well-known phenomenon is referred to as the bias-variance tradeoff.

## Weak Learners
In ensemble learning theory, weak learners, also known as base models, serve as foundational building blocks for constructing more intricate models through their combination. Typically, these base models do not perform optimally in isolation, either due to a high bias resulting from limited degrees of freedom or excessive variance that compromises their robustness, owing to excessive degrees of freedom. The concept behind ensemble methods is to mitigate these shortcomings by aggregating multiple weak learners, thus creating a robust learner or ensemble model capable of achieving superior performance. By leveraging the strengths of individual weak learners and compensating for their weaknesses, the ensemble model aims to strike a balance, effectively reducing bias and/or variance and attaining improved overall performance.

## Combining Weak Learners
When setting up an ensemble learning method, the initial step involves selecting the base models to be aggregated. In many cases, such as in the well-known bagging and boosting methods, a single base learning algorithm is employed to ensure that homogeneous weak learners are utilized, each trained in distinct ways. This yields a resulting ensemble model that is referred to as "homogeneous." Nevertheless, there are alternative methods that employ diverse types of base learning algorithms, resulting in heterogeneous weak learners being combined to form a "heterogeneous ensemble model."

A crucial consideration in this process is to ensure coherence between our choice of weak learners and the aggregation method employed. If we opt for base models with low bias but high variance, the aggregating method should aim to reduce variance. Conversely, if we select base models with low variance but high bias, the aggregating method should be designed to mitigate bias. By aligning the characteristics of the weak learners and the aggregation strategy, we optimize the ensemble's performance and achieve a more effective and balanced model.

In ensemble learning, we encounter three prominent methods: bagging, boosting, and stacking, each with distinct characteristics and objectives.

1. **Bagging**: primarily employs homogeneous weak learners and trains them independently in parallel. It then combines their predictions through a      deterministic averaging process. The emphasis is on reducing variance and enhancing the model's overall robustness.

2. **Boosting**: like bagging, often employs homogeneous weak learners. However, it adopts a sequential and adaptive approach, where each subsequent weak learner's training depends on the performance of the previous ones. The final model combines the predictions using a deterministic strategy, aiming to improve both accuracy and generalization.

3. **Stacking**: on the other hand, embraces heterogeneous weak learners. It trains them in parallel and then leverages a meta-model to blend their predictions effectively. By learning from the outputs of different weak models, stacking aims to create a robust and versatile ensemble model capable of superior performance.

These techniques are related and are crucial to understand as they are valuable in a plethora of circumstances. They are based on the idea that a combination of multiple models produce a more powerful estimator. They are ensemble methods. 

Keep this in mund that In broad terms, bagging primarily aims to construct an ensemble model with reduced variance compared to its individual components. On the other hand, boosting and stacking predominantly strive to create robust models that exhibit lower bias than their constituent models (while also potentially reducing variance). Though variance reduction is an essential consideration for boosting and stacking, their primary emphasis remains on achieving enhanced model strength with minimized bias.


In [39]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


Ensemble models are based on the idea that combining several 'weak' models result in a 'strong' model by combining the predictions. 

An example would be combining several decision trees and take the average prediction. If there are 100 trees, and 60 trees predict *yes*, then the final model will predict *yes*.

Given a complex dataset and a base model (weak learner) we can train the model on the dataset. Often, a single model does not perform well, because it has either a high bias or high variance. The main idea is to try reducing the bias or variance of the base models by combining multiple base models to create an ensemble model that achieves better performance.

For a refresher of the bias-variance tradeoff, you can read the <a href="https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff" target="_blank">wiki</a>

In general, there are three major algorithms that can combine multiple base models:
- Bagging: learns homogeneous base models independently in parallel and combines them through an averaging process.
- Boosting: learns homogeneous base models sequentially.
- Stacking: learns heterogenous base models in parallel and combines them by training a meta-model to output a prediction.

A visualization of the idea (<a href="https://towardsdatascience.com/ensemble-methods-bagging-boosting-and-stacking-c9214a10a205">src</a>)

![img](../Images/ensemble.png)

# Parallel Methods
Bagging stands for bootstrap aggregating and consists of training several models in parallel with replacement. Let's get into boostrapping first.

## Bootstrapping
Let's commence by establishing the concept of bootstrapping. This statistical methodology involves creating sets of samples, each with a size of B (referred to as bootstrap samples), from an initial dataset of size N. This is achieved by randomly selecting B observations with replacement.

Given a dataset $N$, generate bootstrap samples of size $B$ by randomly drawing with replacement $B$ observations.

$$N= \begin{bmatrix} 1 & 2 & 3 \\ 4 & 5 & 6 \\ 7 & 8 & 9 \end{bmatrix}$$

Given $B = 3$

$$\text{Bootstrap samples} = \begin{bmatrix} 1 & 3 & 8 \\ 7 & 6 & 6 \\ ... \\ 9 & 3 & 2 \end{bmatrix}$$

Subject to certain assumptions, these samples exhibit favorable statistical properties. As a preliminary approximation, they can be regarded as drawn directly from the actual underlying (often unknown) data distribution and as independent selections from one another. Thus, they can be treated as representative and uncorrelated samples originating from the genuine data distribution. The hypothesis that must be confirmed to ensure the validity of this approximation are twofold. Firstly, the initial dataset's size N should be sufficiently large to encompass the majority of the underlying distribution's complexity, allowing sampling from the dataset to well approximate sampling from the actual distribution (representativeness). Secondly, the dataset's size N should significantly exceed the bootstrap samples' size B, preventing excessive correlation among samples (independence). Keep in mind that this technique is an approximation.

Bootstrap samples find frequent application, such as in assessing the variance or confidence intervals of statistical estimators. Essentially, a statistical estimator is a function derived from observations, thus constituting a random variable with variance emanating from these observations. To gauge the variance of such an estimator, it becomes necessary to assess it across numerous independent samples drawn from the relevant distribution. However, in many cases, the sheer demand for genuinely independent samples surpasses the available data volume. Consequently, bootstrapping comes into play, generating a series of bootstrap samples which can be deemed as "almost representative" and "almost independent" (almost i.i.d. samples). These bootstrap samples serve as a means to approximate the estimator's variance by computing its value across each of these samples.

## Bagging
When training a model, whether for classification or regression, a function is derived from the training dataset to map inputs to outputs. However, due to the inherent variance in the training dataset—a sample drawn from an underlying distribution—the resulting model exhibits variability. This stems from the fact that a different dataset would yield a different model. Bagging addresses this issue by aiming to create multiple quasi-independent models and averaging their predictions to produce a model with reduced variance. In practice, generating fully independent models is unfeasible due to data limitations. Instead, the approach relies on the favorable "approximate properties" of bootstrap samples, which offer representativeness and a semblance of independence. These bootstrap samples serve as the foundation for fitting models that exhibit a degree of independence, contributing to the overall model's enhanced stability.

To initiate the process, we generate numerous bootstrap samples, where each new sample effectively serves as an additional (nearly) independent dataset derived from the true distribution. Subsequently, a weak learner is trained for each of these samples, and their outcomes are aggregated. This aggregation can be thought of as a form of "averaging" the outputs, resulting in an ensemble model characterized by diminished variance compared to its constituent components.

So, assuming that we have L bootstrap samples of size B denoted

$$
\{z_{1}^{1}, z_{2}^{1}, \ldots, z_{B}^{1}\}, \{z_{1}^{2}, z_{2}^{2}, \ldots, z_{B}^{2}\}, \ldots, \{z_{1}^{L}, z_{2}^{L}, \ldots, z_{B}^{L}\}
$$

$\{z_{b}^{l}\}$ refers to b-th observation of the l-th bootstrap sample. One can fit a model to each of these bootstrap sample to make base models.

$$
w_{1}, w_{2}, \dots, w_{L}
$$

and then aggregate them into some kind of averaging process in order to get an ensemble model with a lower variance. For example, we can define our strong model such that

$$
s_{L} = \frac{1}{L}\sum_{l=1}^{L}w_{l}
$$

is the simple average used in regression problems. Also:

$$
s_{L} = arg_{k}max[card(l|w_{l}=k)]
$$

is used in classification problems and is called hard-voting. The class generated by each model can be viewed as a vote, and the ensemble model returns the class that garners the majority of these votes. In the context of classification problems, another approach involves examining the probabilities assigned to each class by all models. These probabilities are averaged, and the class associated with the highest average probability is selected (referred to as soft-voting). Averages or votes can be either straightforward or weighted, incorporating relevant weights as needed.



In [35]:
X, y = make_classification(n_samples=100, n_features=4,
                          n_informative=2, n_redundant=2,
                          random_state=0, shuffle=False)
clf = BaggingClassifier(estimator=SVC(), n_estimators=10, random_state=0)

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [36]:
clf.fit(X_train, y_train)

BaggingClassifier(estimator=SVC(), random_state=0)

In [37]:
pred = clf.predict(X_test)

In [38]:
accuracy_score(y_test, pred)

1.0

## Random Forest
Ensemble methods commonly employ decision trees as fundamental building blocks, and a collection of such trees is often referred to as a "forest." Within a forest, the constituent trees can be shallow (limited depth) or deep (extended depth, potentially fully grown). While deep trees exhibit lower bias but higher variance, they prove particularly fitting for the bagging method, aimed at variance reduction. The random forest technique, a variant of bagging, employs deep trees trained on bootstrap samples to create an output with decreased variance. Moreover, random forests introduce an additional technique to mitigate inter-tree correlation. During tree growth, rather than solely sampling observations from the dataset to generate bootstrap samples, the approach includes sampling features as well. Only a random subset of features is retained for constructing each tree, enhancing diversity among the fitted trees and diminishing their correlation.

Incorporating feature sampling yields the effect of diversifying the information each tree utilizes for decision-making. Consequently, this practice diminishes the correlation among the distinct output predictions. An additional benefit of feature sampling lies in its ability to enhance the decision-making process's resilience to missing data. Even when observations—whether from the training dataset or external sources—contain missing values, random forests can still perform regression or classification. This is achieved by employing trees that solely consider features with complete data. In summary, the random forest algorithm seamlessly merges the principles of bagging and random feature subset selection to craft more robust and adaptable models.


# Sequential Methods
In sequential methods, the amalgamated weak models are no longer independently trained. Instead, the approach involves an iterative process where model training at a particular step is influenced by the models constructed in preceding steps. Among these methods, "boosting" stands out as the most renowned, yielding an ensemble model that tends to exhibit lower bias compared to the constituent weak learners.

## Boosting
Boosting involves a sequential process of iteratively fitting multiple weak learners. This adaptive approach assigns increasing significance to observations that were inadequately handled by previous models in the sequence. Essentially, each new model focuses on refining its fit to the most challenging instances encountered so far. The outcome is a robust learner with reduced bias, although it's worth noting that boosting can also lead to variance reduction. Like bagging, boosting can be applied to both regression and classification tasks.

With a primary focus on diminishing bias, boosting typically employs base models characterized by low variance and high bias. For instance, when selecting trees as base models, the preference often lies with shallow decision trees possessing minimal depth. Another crucial rationale behind utilizing low variance yet high bias models as weak learners for boosting is their tendency to be less computationally intensive to fit. 

There are two important Boosting algorithms: Adaptive Boosting and Gradient Boosting. these two meta-algorithms diverge in their approach to generating and aggregating weak learners within the sequential process. Adaptive boosting involves updating the weights assigned to each observation in the training dataset, whereas gradient boosting focuses on updating the values of these observations. This primary distinction arises from their respective strategies in tackling the optimization challenge of finding the optimal model, expressed as a weighted sum of weak learners. To continue this section, I will explain the mentioned two algorithms.

## Adaptative boosting
In adaptative boosting (often called “adaboost”), we try to define our ensemble model as a weighted sum of L weak learners

$$
s_{L} = \sum_{l=1}^{L}c_{l}w_{l}
$$

Where $c_{l}$s are the coefficients (base model weights in the final model) and $w_{l}$s are base models. For more mathematical background of the model one can read the following <a href="https://towardsdatascience.com/ensemble-methods-bagging-boosting-and-stacking-c9214a10a205" target="_blank">link</a>.

This method works as follows: At the outset of the algorithm, during the initial stage of the sequence, all observations carry uniform weights of 1/N. The process then iterates L times, corresponding to the L learners in the sequence, following these steps:

1. Fitting the most suitable weak model based on the current observation weights.
    
2. Calculating the update coefficient's value, representing an assessment of the weak learner's performance. This metric guides how significantly the weak learner should influence the ensemble model.

3. Enhancing the strong learner by introducing the new weak learner, scaled by its respective update coefficient.

4. Determining new observation weights that signify the focus for the forthcoming iteration. The weights of incorrectly predicted observations rise, while the weights of correctly predicted observations diminish.

Through the repetition of these steps, a sequence of L models is incrementally developed and amalgamated into a basic linear combination. This combination is characterized by coefficients that indicate the effectiveness of each learner.

## Gradient Boosting
In gradient boosting, the ensemble model we try to build is also a weighted sum of weak learners

$$
s_{L} = \sum_{l=1}^{L}c_{l}w_{l}
$$

The primary distinction from adaptive boosting lies in the formulation of the sequential optimization procedure. In the case of gradient boosting, the approach resembles gradient descent: during each iteration, a weak learner is trained based on the negative gradient of the prevailing fitting error concerning the ongoing ensemble model. 

Consider employing the gradient boosting technique with a designated set of weak models. At the outset of the algorithm (initiating with the first model of the sequence), the pseudo-residuals are initialized to match the observation values. This process unfolds L times (for the L models in the sequence), involving the following steps:

1. Fit the most appropriate weak learner to the pseudo-residuals, approximating the reverse of the gradient concerning the prevailing strong learner.

2. Calculate the optimal step size, determining the magnitude by which the ensemble model is adjusted towards the new weak learner's direction.

3. Enhance the ensemble model by introducing the new weak learner, scaled by the step size (executing a gradient descent step).

4. Compute fresh pseudo-residuals that guide the direction for updating ensemble model predictions for each observation.

Repeating these steps, we have then build sequentially our L models and aggregate them following a gradient descent approach.

For the last point, keep in mind that Gradient boosting can be conceptualized as an generalization of AdaBoost, accommodating a broader scope of differentiable loss functions.

In [41]:
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, y_train)

pred = clf.predict(X_test)
accuracy_score(y_test, pred)

0.96

# Stacking
Stacking diverges from bagging and boosting primarily in two aspects. Firstly, stacking frequently involves the integration of heterogeneous weak learners, encompassing diverse learning algorithms, whereas bagging and boosting predominantly incorporate homogeneous weak learners. Secondly, stacking assimilates the art of merging base models through a meta-model, whereas bagging and boosting employ deterministic algorithms to amalgamate weak learners.

## Stacking process
This ensemble method contains the following steps:

1. Divide the training data into two separate folds.

2. Select L weak learners and train them using data from the first fold.

3. For each of the L weak learners, generate predictions for observations within the second fold.

4. Train the meta-model on the second fold, utilizing the predictions generated by the weak learners as inputs.

In the preceding stages, the dataset was partitioned into two folds, as predictions derived from data utilized in training the weak learners are unsuitable for the meta-model training process. However, this division does present a clear drawback, as only half of the data is available for base model training and the remaining half for meta-model training. To overcome this constraint, a "k-fold cross-training" strategy, akin to the approach in k-fold cross-validation, can be employed. This approach ensures that all observations are utilized for meta-model training, effectively addressing the limitations of the initial split. Doing so, we can produce relevant predictions for each observation of our dataset and then train our meta-model on all these predictions.

## Multi-levels Stacking
A possible extension of stacking is multi-level stacking. It consists in doing stacking with multiple layers. This can be seen as the foundaton of Deep Learning models.

From a pragmatic perspective, it's important to highlight that when selecting a learning algorithm for each meta-model within a multi-level stacking ensemble, there's considerable flexibility in choice—it can encompass a wide range of options, including algorithms employed at lower levels. Additionally, it's worth noting that incorporating more levels can incur either data demands (when a k-folds technique isn't employed, necessitating more data) or time demands (when a k-folds technique is used, leading to the fitting of numerous models). This underscores the balance between data availability and computational efficiency when introducing additional stacking levels.

## Conclusion
Every type of model has its own unique properties that work best with specific kinds of data. Bagging is often used to reduce variance. This can be very handy for noisy data. Boosting is often used to reduce bias. 